## Training Part

In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list the contents of /mydrive
!ls /mydrive

#Navigate to /mydrive/yolov4
%cd /mydrive/Ocular_Disease/

In [ ]:
# Set the path to the directory containing your data
data_dir = '/content/gdrive/MyDrive/Ocular_Disease/train'

## Finding STD and MEAN 

Take and save pictures one by one

In [ ]:
from PIL import Image

# Get the list of all class labels (folders) from the dataset directory
class_labels = os.listdir(data_dir)

# Initialize an empty list to hold the dataset
dataset = []

# Loop through each class label
for class_label in class_labels:
    # Construct the path to the class directory
    class_dir = os.path.join(data_dir, class_label)
    
    # Skip if the current path is not a directory (this ensures we only process folders)
    if not os.path.isdir(class_dir):
        continue

    # Get the list of image files in the class directory
    image_files = os.listdir(class_dir)

    # Loop through each image file in the class directory
    for image_file in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(class_dir, image_file)
        
        # Open the image, convert it to RGB (to ensure it's in the right format)
        image = Image.open(image_path).convert('RGB')
    
        # Append the image and its corresponding class label as a tuple to the dataset
        dataset.append((image, class_label))

In [ ]:
# Initialize arrays to store the mean and standard deviation for each channel (RGB)
mean = np.zeros(3)  # To store mean for each RGB channel
std = np.zeros(3)   # To store standard deviation for each RGB channel
count = 0

# Iterate over the dataset and compute the mean
for image, _ in dataset:
    # Convert the image from PIL format to a NumPy array
    image_array = np.array(image)  
    height, width, _ = image_array.shape  # Get the image dimensions (height, width, channels)
    
    # Reshape the image array to a 2D array (height * width, 3) for easier processing of channels
    reshaped_image_array = image_array.reshape(height * width, 3)
    
    # Calculate the mean across each channel (R, G, B) and add it to the running mean
    mean += np.mean(reshaped_image_array, axis=0)
    
    count += 1  # Increment the image counter

# Divide the sum of means by the number of images to get the average mean for each channel
mean /= count

# Iterate over the dataset again to compute the standard deviation
for image, _ in dataset:
    # Convert the image to a NumPy array
    image_array = np.array(image)
    height, width, _ = image_array.shape  # Get the dimensions of the image
    reshaped_image_array = image_array.reshape(height * width, 3)  # Reshape the image array
    
    # Calculate the squared difference from the mean for each pixel and channel
    std += np.mean((reshaped_image_array - mean) ** 2, axis=0)

# Compute the square root of the variance (std) to get the standard deviation
std = np.sqrt(std / count)

# Output the computed mean and standard deviation for each RGB channel
print('Mean:', mean)
print('Standard Deviation:', std)


Mean: [112.121926    71.96017866  38.72672353]
Standard Deviation: [75.27826614 54.42092274 37.71428029]


## Continue

In [ ]:
# Define the transformations to be applied to the images
transform = transforms.Compose([
    # Resize the image to a target size of (224, 224) pixels
    transforms.Resize((224, 224)),

    # Convert the image to a tensor (this converts the image from PIL to a tensor format)
    transforms.ToTensor(),

    # Normalize the image tensor with pre-defined mean and standard deviation values
    # These values are commonly used for models like ResNet that are pre-trained on ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

In [ ]:
# Create the ImageFolder dataset
dataset = ImageFolder(data_dir, transform=transform)

print("Number of samples in the training set:", len(dataset))

## Data Augmentation Part

In [ ]:
# Define the transforms for data augmentation
augmentation_transform = transforms.Compose([
    transforms.RandomResizedCrop(224), # Apply random resized crop to the image, cropping and resizing it to 224x224 pixels
    transforms.RandomHorizontalFlip(), # Randomly flip the image horizontally with a probability of 0.5
    transforms.RandomRotation(10), # Apply random rotation between -10 and 10 degrees
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), # Apply random adjustments to brightness, contrast, saturation, and hue
    transforms.ToTensor(), # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize the image using the mean and std of ImageNet
])

# Apply the defined augmentation transforms to the dataset using ImageFolder
augmented_dataset = ImageFolder(data_dir, transform=augmentation_transform)

print("Number of samples in the training set:", len(augmented_dataset))

## Combining Two Dataset

In [ ]:
from torch.utils.data import ConcatDataset

# Combine the original dataset and the augmented dataset
combined_dataset = ConcatDataset([dataset, augmented_dataset])

print("Number of samples in the training set:", len(combined_dataset))

### Combined Dataset

In [ ]:
# Split the dataset into training, validation, and test sets
train_ratio = 0.8
val_ratio = 0.2


# Split the dataset into training and validation sets
train_dataset, val_dataset = train_test_split(combined_dataset, test_size=val_ratio, random_state=42)

# Optionally, you can print the number of samples in each set
print("Number of samples in the training set:", len(train_dataset))
print("Number of samples in the validation set:", len(val_dataset))

### Normal Dataset

In [ ]:
# Split the dataset into training, validation, and test sets
train_ratio = 0.8
val_ratio = 0.2


# Split the dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=val_ratio, random_state=42)

# Optionally, you can print the number of samples in each set
print("Number of samples in the training set:", len(train_dataset))
print("Number of samples in the validation set:", len(val_dataset))

### BatchSize

In [ ]:
# Create DataLoaders for training, validation, and test sets
batch_size = 64  # Adjust the batch size according to your needs
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

### MobileNet

In [ ]:
# Get the unique class names from the files
class_names = sorted(os.listdir(data_dir))
num_classes = len(class_names)

In [ ]:
# Load the pretrained MobileNet model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

In [ ]:
# Modify the final classifier to match the number of classes (diseases)
model.classifier = nn.Sequential(
    # Dropout layer with 0.2 probability to help prevent overfitting
    nn.Dropout(0.2),

    # Linear layer that maps from 1280 features (e.g., from EfficientNet) to 'num_classes' outputs
    nn.Linear(1280, num_classes)  # Adjust 1280 if your model has a different feature size before classification
)

In [ ]:
model = model.to('cuda:0')

In [ ]:
# Move the model to the desired device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

### VGG19

In [ ]:
# Load the pretrained VGG19 model
model = models.vgg19(pretrained=True)

In [ ]:
# Freeze the parameters of all layers except the final classifier
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Replace the final fully connected layers for disease classification
num_classes = len(dataset.classes)
model.classifier = nn.Sequential(
    # First fully connected layer, reducing 25088 input features (e.g., from VGG) to 4096
    nn.Linear(25088, 4096),  # Adjust the input size (25088) if needed based on your model architecture
    nn.ReLU(),  # ReLU activation for introducing non-linearity
    nn.Dropout(p=0.5),  # Dropout layer for regularization to avoid overfitting
    
    # Second fully connected layer, from 4096 to 4096 features (same size)
    nn.Linear(4096, 4096),
    nn.ReLU(),  # Another ReLU activation
    nn.Dropout(p=0.5),  # Dropout again to help with generalization

    # Final fully connected layer that maps 4096 features to the number of classes
    nn.Linear(4096, num_classes)  # 'num_classes' corresponds to the number of disease classes
)

In [ ]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

### Class Imbalance Solution

In [ ]:
import torch.optim as optim

# Assuming you have a PyTorch dataset called 'dataset' containing your data

# Compute class weights
class_counts = [350, 200, 101, 296, 140, 90, 468, 150]  # Example class counts (replace with your own)
total_samples = sum(class_counts)
class_weights = [total_samples / (len(class_counts) * count) for count in class_counts]
class_weights = torch.Tensor(class_weights).to(device)  # Move class weights to the desired device

In [ ]:
criterion = nn.CrossEntropyLoss(weight=torch.Tensor(class_weights))
optimizer = optim.Adam(model.parameters(), lr=5e-3)  # Replace with your chosen optimizer

### SGD

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)

### Adam

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

## Training

In [ ]:
# Training loop parameters
num_epochs = 100  
train_loss_list = []  
val_loss_list = []    
train_acc_list = []   
val_acc_list = []     

# Get the total number of steps in the train and validation dataloaders
total_train_steps = len(train_dataloader)
total_val_steps = len(val_dataloader)

# Start the training loop
for epoch in range(num_epochs):
    # --- Training Phase ---
    model.train()  # Set the model to training mode
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Loop over the training data
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)  # Move images to the GPU
        labels = labels.to(device)  # Move labels to the GPU

        # Forward pass
        outputs = model(images)  # Get model predictions
        loss = criterion(outputs, labels)  # Calculate the loss

        # Backward pass and optimization step
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update model weights

        # Accumulate loss and accuracy
        train_loss += loss.item()  # Add loss for the current batch
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        train_total += labels.size(0)  # Total number of labels in this batch
        train_correct += (predicted == labels).sum().item()  # Count correct predictions

        # Print training progress every 10 steps
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_train_steps}], '
                  f'Training Loss: {train_loss / (i + 1):.4f}')

    # Calculate average training loss and accuracy for the epoch
    train_loss /= total_train_steps
    train_acc = 100 * train_correct / train_total

    # --- Validation Phase ---
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # Disable gradient tracking for validation
        for i, (images, labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)  # Get model predictions
            loss = criterion(outputs, labels)  # Calculate the validation loss

            val_loss += loss.item()  # Accumulate validation loss
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class
            val_total += labels.size(0)  # Total number of labels in this batch
            val_correct += (predicted == labels).sum().item()  # Count correct predictions

    # Calculate average validation loss and accuracy for the epoch
    val_loss /= total_val_steps
    val_acc = 100 * val_correct / val_total

    # Store the metrics for later use
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, '
          f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')


In [ ]:
import matplotlib.pyplot as plt

# Plotting training and validation results
plt.figure(figsize=(10, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_loss_list, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_loss_list, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_acc_list, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_acc_list, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Adjust layout for better presentation
plt.tight_layout()
# Show the plot
plt.show()


In [ ]:
# Save the fine-tuned model
torch.save(model.state_dict(), './fine_tuned_vgg19.pth')

## Test Part

In [ ]:
from torchvision.models import vgg19

In [ ]:
# Load the fine-tuned VGG19 model
model = vgg19(pretrained=False)  # Initialize VGG19 without pretrained weights
num_classes = 8

# Modify the classifier to match the number of classes for the task (8 in this case)
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # Fully connected layer with input size 25088 and output size 4096
    nn.ReLU(),  # ReLU activation function
    nn.Dropout(p=0.5),  # Dropout layer with 50% probability for regularization
    nn.Linear(4096, 4096),  # Fully connected layer with input size 4096 and output size 4096
    nn.ReLU(),  # ReLU activation function
    nn.Dropout(p=0.5),  # Dropout layer with 50% probability for regularization
    nn.Linear(4096, num_classes)  # Final fully connected layer to match the number of output classes
)

# Load the fine-tuned weights from a saved checkpoint
model.load_state_dict(torch.load('./fine_tuned_vgg19.pth'))

# Set the model to evaluation mode (disables dropout and batch normalization for inference)
model.eval()


In [ ]:
# Define the transformations to be applied to the input image
transform = transforms.Compose([  # Combine multiple transformations into a single pipeline
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels (common input size for VGG19)
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image using the mean and std of ImageNet
    # These normalization values are typical for pre-trained models like VGG19
])

### To check the image by image

In [ ]:
# Load and preprocess the test image
image_path = '/content/gdrive/MyDrive/Ocular_Disease/classes/image.jpg'  # Replace with the path to your test image
image = Image.open(image_path).convert('RGB')  # Open the image and convert it to RGB mode
image_tensor = transform(image).unsqueeze(0)  # Apply the defined transformations and add a batch dimension (unsqueeze(0))

In [ ]:
# Perform the inference
with torch.no_grad():  # Disable gradient calculation as it's not needed for inference
    outputs = model(image_tensor)  # Perform forward pass through the model to get raw outputs
    _, predicted = torch.max(outputs, 1)  # Get the index of the class with the highest score (prediction)

In [ ]:
# Map the predicted class index to the corresponding class label
predicted_label = class_labels[predicted.item()]

In [ ]:
# Print the predicted label
print('Predicted class:', predicted_label)

### To check the whole file

In [ ]:
class_labels = ['OTHER', 'age-related macular degeneration', 'cataract', 'diabetic_retinopathy', 'glaucoma', 'hypertensive retinopathy', 'normal_fundus', 'pathological myopia' ]

In [ ]:
# Set the path to the directory containing your test images
test_dir = '/content/gdrive/MyDrive/Ocular_Disease/outputs/diabetes'  # Replace with the path to your test directory

# Get a list of image file names in the test directory
image_files = os.listdir(test_dir)

# Perform inference for each image in the directory
for image_file in image_files:
    # Load and preprocess the test image
    image_path = os.path.join(test_dir, image_file)  # Construct the full path to the image file
    image = Image.open(image_path).convert('RGB')  # Open and convert the image to RGB format
    image_tensor = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension

    # Perform the inference
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(image_tensor)  # Perform a forward pass through the model to get predictions
        _, predicted = torch.max(outputs, 1)  # Get the index of the class with the highest score (prediction)

    # Map the predicted class index to the corresponding class label
    predicted_label = class_labels[predicted.item()]  # Convert the index to the class label

    print(f'Image: {image_file}, Predicted class: {predicted_label}')  # Output the result for each image